In [ ]:
import time

def time_function(func, n):
    start = time.time()
    result = func(n)
    end = time.time()
    print(f"{func.__name__}({n}) = {result}, Time: {end - start:.6f} seconds")

In [ ]:
from collections import defaultdict
counter = defaultdict(int)

# Fibonacci recursive
def fibonacci_recursive(n):
    # print(f"Calculating Fibonacci({n})")
    counter[n] += 1

    if n <= 1:
        return n
    return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

print(fibonacci_recursive(25))

total_calls = sum([count for count in counter.values()])
print(counter)
print(total_calls)

# Fibonacci tabulation
def fibonacci_dp(n):
    # Initialising the array to store subproblem results
    fib = [0] * (n + 1)
    
    # Base cases
    fib[0] = 0
    if n > 0:
        fib[1] = 1
    
    # Filling the array
    for i in range(2, n + 1):
        fib[i] = fib[i-1] + fib[i-2]
    
    return fib[n]

# Fibonnaci memoization
def fibonacci_memo(n: int, memo: dict = None) -> int:
    if memo is None:
        memo = {}
    
    if n in memo:
        return memo[n]
    
    if n <= 1:
        return n
    
    memo[n] = fibonacci_memo(n-1, memo) + fibonacci_memo(n-2, memo)
    return memo[n]

In [ ]:
time_function(fibonacci_recursive, 42)
time_function(fibonacci_dp, 42)
time_function(fibonacci_memo, 42)

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Item:
    weight: float
    value: float

@dataclass
class Rucksack:
    capacity: int = 25
    items: List[Item] = field(default_factory=lambda: [])

    def total_value(self) -> float:
        return sum([item.value for item in self.items])

    def total_weight(self) -> float:
        return sum([item.weight for item in self.items])

    def remaining_capacity(self) -> float:
        return self.capacity - self.total_weight()
    
    def add_item(self, item: Item):
        if self.remaining_capacity() < item.weight:
            print("Cannot exceed maximum capacity.")
            return
        self.items.append(item)

    def get_info(self):
        print("Remaining capacity:", self.remaining_capacity())
        print("Total value:", self.total_value())
        print("Total weight:", self.total_weight())

@dataclass
class FractionalItem(Item):
    weight: float
    value: float

    def to_fraction(self, fraction: float):
        if fraction > 1: raise Exception("Illegal fraction")
        return FractionalItem(self.weight * fraction, self.value * fraction)

In [ ]:
# Fractional Backpack problem
def fractional_backpack_problem(capacity: int, items: List[FractionalItem]):
    items_by_ratio = sorted(items, key=lambda item: item.value / item.weight, reverse=True)
    
    rucksack = Rucksack(capacity)

    for item in items_by_ratio:
        remaining_capacity = rucksack.remaining_capacity()
        
        if remaining_capacity == 0:
            break
        
        if remaining_capacity >= item.weight:
            rucksack.add_item(item)
        else:
            fraction = remaining_capacity / item.weight
            fractional_item = item.to_fraction(fraction)
            rucksack.add_item(fractional_item)

    return rucksack

In [ ]:
if __name__ == "__main__":
    prizes = [
        FractionalItem(22, 19),
        FractionalItem(10, 9),
        FractionalItem(9, 9),
        FractionalItem(7, 6),
    ]

    fractional_backpack = fractional_backpack_problem(25, prizes)
    
    print("Fractional Backpack:", fractional_backpack)
    fractional_backpack.get_info()

In [ ]:
# 0 / 1 Backpack problem - Dynamic Programming approach
def backpack_dp(backpack: Rucksack, available_items: List[Item]) -> Rucksack:
    capacity = backpack.capacity
    n = len(available_items)
    
    # Matrix width=capacity+1 & height=available_items+1
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]
    
    # Fill the DP table.
    # We start from 2nd row (index: 1) as the first row is already filled with 0 for 0-case item
    for i in range(1, n + 1):
        # Current item (0-indexed)
        item = available_items[i - 1]  
        
        for w in range(capacity + 1):
            dp[i][w] = dp[i - 1][w]
            
            # If the item fits - take the current item.
            if item.weight <= w:
                value_with_item = dp[i - 1][w - item.weight] + item.value
                dp[i][w] = max(dp[i][w], value_with_item)
    

    # Backtrack to find which items were selected.
    selected_items = []
    i, w = n, capacity
    
    while i > 0 and w > 0:
        # If value change comes from including current item, we include the item.
        if dp[i][w] != dp[i - 1][w]:
            item = available_items[i - 1]
            selected_items.append(item)
            w -= item.weight
        i -= 1
    
    # max_value = dp[n][capacity]

    for item in selected_items:
        backpack.add_item(item)

    return backpack

In [ ]:
if __name__ == "__main__":
    backpack = Rucksack(25)

    prizes = [
        Item(weight=22, value=19),
        Item(weight=10, value=9),
        Item(weight=9, value=9),
        Item(weight=7, value=6),
    ]

    optimal_backpack = backpack_dp(backpack, prizes)

    print("Zero One Backpack:", optimal_backpack)
    optimal_backpack.get_info()

In [ ]:
if __name__ == "__main__":
    backpack = Rucksack(8)

    prizes = [
        Item(weight=2, value=1),
        Item(weight=3, value=2),
        Item(weight=4, value=5),
        Item(weight=5, value=6),
    ]

    optimal_backpack = backpack_dp(backpack, prizes)

    print("Zero One Backpack:", optimal_backpack)
    optimal_backpack.get_info()

In [ ]:
import sys
from typing import List, Tuple

def shortest_path(graph: List[List[Tuple[int, int]]], start: int, end: int) -> int:
    n = len(graph)
    distances = [sys.maxsize] * n
    distances[start] = 0
    
    for _ in range(n - 1):
        for u in range(n):
            for v, weight in graph[u]:
                if distances[u] != sys.maxsize and distances[u] + weight < distances[v]:
                    distances[v] = distances[u] + weight
    
    return distances[end]

# Example graph: [[(neighbour, weight), ...], ...]
graph = [
    [(1, 4), (2, 1)],
    [(3, 1)],
    [(1, 2), (3, 5)],
    [(4, 3)],
    []
]

print(shortest_path(graph, 0, 4))
